Hosting the qdrant in local machine

docker pull qdrant/qdrant
docker run -p 6333:6333 -p 6334:6334 \
    -v $(pwd)/qdrant_storage:/qdrant/storage:z \
    qdrant/qdrant


In [3]:
import qdrant_client
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.document_loaders import DirectoryLoader
from langchain_qdrant import Qdrant

In [23]:
url = "http://localhost:6333"

client = QdrantClient(url=url)

model_name = "thenlper/gte-large"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/Users/sarathyashu/Desktop/Learnings/workbook/paris_env/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# load files from directory
loader = DirectoryLoader('/Users/sarathyashu/Desktop/Learnings/workbook/data', glob="**/*.txt")
docs = loader.load()
# semantic chunker
textsplitter = SemanticChunker(hf)
splitted_docs = textsplitter.transform_documents(docs)

In [30]:
qdrant = Qdrant.from_documents(documents=splitted_docs, embedding=hf,
    url = url,
    prefer_grpc=True,
    collection_name="test_documents",
)

In [31]:
client.count(collection_name='test_documents')

CountResult(count=26)

In [36]:
response = qdrant.similarity_search(query='what is state of union', k=5)

In [37]:
response

[Document(page_content='We can do this. My fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy. In this Capitol, generation after generation, Americans have debated great questions amid great strife, and have done great things. We have fought for freedom, expanded liberty, defeated totalitarianism and terror. And built the strongest, freest, and most prosperous nation the world has ever known. Now is the hour. Our moment of responsibility. Our test of resolve and conscience, of history itself. It is in this moment that our character is formed. Our purpose is found. Our future is forged. Well I know this nation. We will meet the test. To protect freedom and liberty, to expand fairness and opportunity. We will save democracy. As hard as these times have been, I am more optimistic about America today than I have been my whole life. Because I see the future that is within our grasp. Because I know there is simply nothing beyond our capacity. We are th